In [1]:
import os

In [2]:
%pwd

'c:\\Users\\H P\\Desktop\\DeepLearningProjects\\Chicken_Disease_Classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\H P\\Desktop\\DeepLearningProjects\\Chicken_Disease_Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from CnnClassifier.constants import *
from CnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from CnnClassifier import logger
from CnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of Size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_filepath: str
        Extracts the zip file into the Data Directory
        Functions return None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-17 20:05:17,999: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-17 20:05:18,006: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-17 20:05:18,009: INFO: common: created directory at: artifacts]
[2025-07-17 20:05:18,013: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-17 20:05:22,695: INFO: 3299834989: artifacts/data_ingestion/data.zip download with following info: 
Connection: close
Content-Length: 13605616
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "087b28c83aeb2357d013508bbaea73abf0d1786171927d950debfd0f07ab0e7c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1F62:149E5E:B70F33:D0360A:68792CB4
Accept-Ranges: bytes
Date: Thu, 17 Jul 2025 17:05:20 GMT
Via: 1.1 varnish
X-Served-By: cache-mrs10582-MRS
X-Cache: H